函数装饰器用于在源码中“标记”函数，以某种方式增强函数的行为。

除了在装饰器中有用处之外，闭包还是会掉式异步编程和函数式编程风格的基础。

装饰器是可调用的对象，其参数是另一个函数（被装饰的函数/类）。装饰器可能会处理被装饰的函数，然后将其返回，或者替换为另一个函数或可调用对象。

严格来说，装饰器只是语法糖，元编程（在运行时改变程序的行为）时尤其方便，其特性：
- 能把被装饰的函数替换成其他函数
- 在加载模块时立即执行（导入时）

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    
    # 返回 inner 函数对象
    return inner

# 使用 demo 装饰 target
@deco
def target():
    print('running target()')

target()

In [2]:
target

In [3]:
# 保存被 @register 装饰的函数引用
registry = []

# 参数是一个函数
def register(func):
    # 打印被装饰函数
    print('running register(%s)' % func)
    
    # 存入 registry
    registry.append(func)
    
    # 必须返回函数
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

# 当作脚本运行时执行
if __name__=='__main__':
    main()

导入时和运行时的区别
- 装饰器在导入模块时立即执行
- 被装饰的函数只在明确调用时运行


装饰器通常在一个模块中定义，然后应用到其他模块中的函数上
大多数装饰器会在内部定义一个函数，然后将其返回，替换被装饰得函数
- 使用内部函数的代码几乎都要靠闭包才能正确运行

很多 Python Web 框架使用装饰器把函数添加到某种中央注册处，这种注册装饰器可能会也可能不会修改被装饰的函数

In [4]:
import registration

In [5]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # 上下文

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


# 起初为空列表
promos = []

# 把函数添加到 promos 列表中
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion  # <3>
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):  # <4>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

In [6]:
# 读取一个局部变量和全局变量
def f1(a):
    print(a)
    print(b)

f1(3)

NameError: name 'b' is not defined

In [7]:
# 函数的定义体中给 b 赋值，因此是局部变量
# Python 假定在函数定义体中赋值的变量是局部变量
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [8]:
# 
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [9]:
b

9

In [10]:
f3(3)

3
9


In [11]:
b = 30
b

30

闭包指延伸了作用域的函数，其中包含函数定义体重的引用、但是不在定义体中定义的非全局变量
- 能访问定义体之外定义的非全局变量

In [12]:
# 计算移动平均值的类
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Averager()
avg(10)

10.0

In [13]:
avg(11)

10.5

In [14]:
avg(12)

11.0

In [15]:
# 计算移动平均值的高阶函数
def make_averager():
    # 自由变量（free variable），未在本地作用域中绑定的变量
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

In [18]:
# 在 __code__ 属性中保存局部变量和自由变量的名称
# 局部变量
avg.__code__.co_varnames

('new_value', 'total')

In [19]:
# 自由变量
avg.__code__.co_freevars

('series',)

In [20]:
# series 的绑定
# __closure__ 各个元素对应于 __code__.co_freevars 中的名称
avg.__closure__

(<cell at 0x0000025FD63DEE50: list object at 0x0000025FD6346440>,)

In [21]:
# 保存真正的值
avg.__closure__[0].cell_contents

[10, 11, 12]

闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用于不可用，但仍然能使用那些绑定

只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量

In [22]:
# 计算移动平均值的高阶函数，不保历史数据
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

对数字、字符串、元组等不可变类型来说，只能读取，不能更新。

Python 3 引入了 nonlocal 声明，它的作用是把变量标记为自由变量，即使在函数中为变量赋予新值，也会变成自由变量。
如果为 nonlocal 声明的变量赋予新值，闭包中保存的绑定会更新。

In [23]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

10.0

In [24]:
# 装饰器，输出函数的运行时间
import time

def clock(func):
    
    # 定义内部函数，接受任意个位置参数
    def clocked(*args):
        
        # 起始时间
        t0 = time.time()
        
        # 执行函数，clocked 闭包中包含自由变量 func
        result = func(*args)
        
        # 经过时间
        elapsed = time.time() - t0
        
        # 函数名称
        name = func.__name__
        
        # 打印字符串
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        
        # 返回函数执行结果
        return result
    
    return clocked

In [25]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12337255s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720


In [26]:
# 装饰器，输出函数的运行时间
# 支持处理关键字参数
import time
import functools

def clock(func):
    
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        
        # 起始时间
        t0 = time.time()
        
        # 执行函数，clocked 闭包中包含自由变量 func
        result = func(*args, **kwargs)
        
        # 经过时间
        elapsed = time.time() - t0
        
        # 函数名称
        name = func.__name__
        
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        
        # 打印字符串
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        
        # 返回函数执行结果
        return result
    
    return clocked

Python 内置了三个用于装饰方法的函数：property、classmethod、staticmethod

另一个常见的装饰器时 functools.wraps ，它的作用是协助构建行为良好的装饰器

In [27]:
# 递归生成第 n 个斐波那契数
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00240016s] fibonacci(4) -> 3
[0.00240016s] fibonacci(5) -> 5
[0.00240016s] fibonacci(6) -> 8
8


functools.lru_cache 实现了备忘（memorization）功能，把耗时的函数结果保存起来，避免传入相同的参数时重复计算。

LRU 是 Least Recently Used 的缩写，一段时间不用的缓存条目会被扔掉

functools.lru_cache(maxsize=128, typed=False)
- maxsize：2 的幂，指定存储多少个调用结果
- typed：把不同参数类型得到的结果分开存放

lru_cache 使用字典存储结果，而且键根据调用时传入的位置参数和关键字参数创建，所以被 lru_cache 装饰的函数，它的所有参数都必须是 **可散列** 的。

In [28]:
import functools

# lru_cache 应用到 clock 返回的函数上
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00000000s] fibonacci(6) -> 8
8


In [29]:
import html

# 适用于任何 Python 类型
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [30]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [31]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [32]:
htmlize(42)

'<pre>42</pre>'

In [33]:
htmlize(['alpha', 66, {3, 2, 1}])

'<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>'

使用 @singledispatch 装饰的普通函数会变成泛函数（generic function）：根据第一个参数的类型，以不同方式执行相同操作的一组函数。
- 泛函数也叫做多方法

singledispatch 创建一个自定义的装饰器，把多个函数绑定在一起组成一个泛函数

只要可能，注册的专门函数应该处理抽象基类，这样代码的兼容类型更广泛
使用抽象基类检查类型，可以让代码支持这些抽象基类现有和未来的具体子类或虚拟子类

singledispatch 机制的特征是可以在系统的任何地方和任何模块中注册专门函数，还可以为不是自己编写的或不能修改的类添加自定义函数

@singledispatch 的优点是支持模块化扩展：各个模块可以为它支持的各个类型注册一个专门函数

In [34]:
from functools import singledispatch
from collections import abc
import numbers
import html

# 基函数
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

# 专门函数
@htmlize.register(str)
def _(text):  # 函数名可省略
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

# 专门函数
@htmlize.register(numbers.Integral)  # int 的虚拟超类
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

# 专门函数
# 叠放多个 register 装饰器，让同一个函数支持不同类型
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [35]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [36]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [37]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [38]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [39]:
htmlize(['alpha', 66, {3, 2, 1}])

'<ul>\n<li><p>alpha</p></li>\n<li><pre>66 (0x42)</pre></li>\n<li><pre>{1, 2, 3}</pre></li>\n</ul>'

解析源码中的装饰器时，Python 把被装饰的函数作为第一个参数传递给装饰器函数

```python
@d1
@d2
def f():
    print('f')

# 等同于
def f():
    print('f')
f = d1(d2(f))
```

让装饰器接受其他参数：创建一个装饰器工厂函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上

In [40]:
registry = set()  # <1>

# 装饰器工厂函数
def register(active=True):
    
    # 装饰器
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # 闭包
            registry.add(func)
        else:
            registry.discard(func)

        return func  # 装饰器返回函数
    return decorate  # 装饰器工厂函数返回 decorate

# 必须作为函数调用，并传入所需参数
@register(active=False)
def f1():
    print('running f1()')

# 关键字参数默认值
@register()
def f2():
    print('running f2()')

# 无装饰器
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x0000025FD62FF9D0>)
running register(active=True)->decorate(<function f2 at 0x0000025FD62FF280>)


In [41]:
# 只有 f2 函数在 registry 中
from registration_param import *

running register(active=False)->decorate(<function f1 at 0x0000025FD63E30D0>)
running register(active=True)->decorate(<function f2 at 0x0000025FD63E3EE0>)


In [42]:
# 导入模块时，f2 在 registry 中
registry

{<function registration_param.f2()>}

In [43]:
# register() 返回 decorate，应用到 f3 上
#添加到 registry 中
register()(f3)

running register(active=True)->decorate(<function f3 at 0x0000025FD633D940>)


<function registration_param.f3()>

In [44]:
registry

{<function registration_param.f2()>, <function registration_param.f3()>}

In [45]:
# 从 registry 中删除 f2
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x0000025FD63E3EE0>)


<function registration_param.f2()>

In [46]:
registry

{<function registration_param.f3()>}

In [47]:
# 参数化 clock 装饰器
# 传入一个格式字符串，控制被装饰函数的输出

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

# 装饰器工厂函数
def clock(fmt=DEFAULT_FMT):
    
    # 装饰器
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)  # 被装饰函数返回的真正结果
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # args 是用于显示的字符串
            result = repr(_result)  # 字符串表示形式
            print(fmt.format(**locals()))  # 在 fmt 中引用 clocked 局部变量
            return _result
        return clocked
    return decorate

In [48]:
# 不传参调用，默认格式
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12326908s] snooze(0.123) -> None
[0.12300181s] snooze(0.123) -> None
[0.12364721s] snooze(0.123) -> None


In [49]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.1238241195678711s
snooze: 0.12393784523010254s
snooze: 0.12326216697692871s


In [50]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.124s


掌握闭包和 nonlocal 不仅对构建装饰器有帮助，还能协助构建 GUI 程序时面向事件编程，或者使用回调处理异步 I/O 。

wrapt 模块的作用是简化装饰器和动态函数包装器的实现，即使多层装饰也支持内省，而且行为正确，即可以应用到方法上，也可以作为描述符使用。

decorator 包旨在简化使用装饰器的方式，并且通过各种复杂的示例推广装饰器。